# Merge and Groupby
Duncan Callaway

This notebook gives an introduction to using Pandas' `merge` and `groupby` methods.  

In [ ]:
import pandas as pd
import numpy as np

## Row and column labels
The columns are identified with a list of values.  Let's look at the fruit data set again:

In [ ]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
fruit_info_df

### Q: How do I print out just the columns?

In [ ]:
fruit_info_df.columns

### Q: And the rows?

The rows are similarly labeled:

In [ ]:
fruit_info_df.index

## Merging
Lets make another data frame and tack it on to the first

In [ ]:
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})
price_df

Now let's blindly merge:

In [ ]:
pd.merge(price_df,fruit_info_df)

What went wrong?

First, we didn't spell fruit correctly.  Two ways to fix.  First, specify the columns directly:

In [ ]:
pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit')

Second, fix the spelling and *don't* tell pandas.  In this case pandas works to figure out what's in common.

In [ ]:
price_df.columns[0]='fruit'

Bummer!  Can't mutate index values.  What to do?

In [ ]:
col_list = list(price_df.columns)
col_list

In [ ]:
col_list[1] = 'fruit'

In [ ]:
price_df.columns = col_list
price_df

In [ ]:
pd.merge(fruit_info_df,price_df)

Note we can use different syntax:

In [ ]:
fruit_info_df.merge(price_df)

### Q: Now we're still missing raspberries -- why?

Again, spelling error in the new frame.  Let's fix:

In [ ]:
price_df.loc[3,'fruit'] = 'raspberry'

Note we could change individual entries in the data frame itself.  They are mutable.

In [ ]:
fruit_info_df.merge(price_df)

Another few things to takeaway from this
1. Merge can be brutal.  That is, it'll drop data without telling you.  BUT that's if we use the default 'inner' merge.  In a few lecture we'll talk about alternative ways to merge that are a little less draconian.
2. It's important to review your results.  How many rows do you expect?  How many do you actually get?  Did something important get chucked out?  The ensuing solutions are the non-glamorous tasks of data cleaning.

Note, there are other commands -- `join`, `concat`, and these do similar things to `merge`.

I've found merge seems to work well for most purposes.

FWIW, `pd.concat` seems to be a little more brute force -- requires more careful syntax, but likely does unexpected things less often once you understand the syntax.

In [ ]:
merged_df = fruit_info_df.merge(price_df)
merged_df

We can streamline by replacing the index number with the fruit column.  

### Q: in the following, what's the `inplace` command for?  

In [ ]:
merged_df.set_index('fruit', inplace = True)
merged_df

### A: It means the re-defined dataframe is assigned to the original name.  
This is advantageous in memory constrained situations.  

## Multilevel indexing 

We can also assign "multilevel" column or row names, like so:

In [ ]:
levels = [('categorical', 'color'),('quantitative', 'weight'),('quantitative','price')]
levels

Note the  use of tuples (sets of values in parentheses) in setting up multiindex.  This will come again later.  

In [ ]:
merged_df.columns = pd.MultiIndex.from_tuples(levels)
merged_df

Now we have categories and subcategories of columns:

In [ ]:
merged_df['quantitative']

### Q: How can we get data from an individual column?
Aim to get the `weight` column:

In [ ]:
merged_df[('quantitative','weight')]

## Advanced multilevel (did not do in lecture) 
Note, we can also drop and add things.  With multilevel indexing things get a little tricky.  

First, we can drop everything from the top level:

In [ ]:
merged_test_df = merged_df.drop(columns=[('quantitative',)], axis = 1)
merged_test_df

Note that I put the column identifier inside the parens, like a tuple, but it's not essential there.

However if we want to drop only a column from the second level, we get an error without the tuple syntax:

In [ ]:
merged_test_df = merged_df.drop(columns=[('quantitative','price')], axis = 1)
merged_test_df

We can also drop rows: 

In [ ]:
merged_df.drop(index=[('apple')], axis = 0, inplace = True)
merged_df

Note indexing multilevels with `.loc` gets a little tricky.  The thing to keep in mind is that you're working with tuples in each index location:

In [ ]:
merged_df.loc['banana', ('quantitative', 'price')]

If you leave an entry of the tuple empty you get all values.  

In [ ]:
merged_df.loc['banana', ('quantitative', )]

You can also loop through the columns of the multilevel data frame like this: 

In [ ]:
for i, j in merged_df:
    print(merged_df.loc['banana', (i, j)])

Some added thoughts:
1. Multilevel indexing works for columns and index
2. It can be a powerful way to summarize your data and quickly reference subsets of it.
4. However it can also be a colossal pain in the rear -- indexing with multilevel is often very hard to parse and debug.

## Groupby

First, let's have another look at today's power point file.  Now we'll learn about how groupby works.

Back to the notebook, let's make a toy DF (example taken from Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653):

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

Let's group just the `data1` column by the `key1` column. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that.  

Note, the syntax is to begin by invoking the portion of the dataframe we want to group (here, `df['data1']`), then we apply the groupby method with the portion of hte dataframe we want to group on (here `df['key1']`)

What is the object that results?

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

As we see, it's not simply a new DataFrame. Instead, it's an object, in this case `SeriesGroupBy`.  We'll see in a moment that if we group many columns of data we get a `DataFrameGroupBy` object.

To look inside we need to use different syntax.  The specific thing we're looking for are the groups of the object...but let's tab in to the grouped object to see what's there.

In [ ]:
grouped.groups

That gave us the groups (a and b) and the indices of elements in the groups, but nothing else. 

You can see this structure looks like a dict.  a and b are the keys, and the data are lists associated with each key -- the values.  

But the `grouped` object is capable of making computations across all groups -- this is where it gets powerful.   

We can try things like `sum`, `min` and `max`.

Notice if you don't put the parens after the method, pandas returns information about what the method does, but not it's actual output.  

In [ ]:
grouped.sum()

You can also pass `numpy` functions into the aggregate command.

But it can be informative to look at what's inside. We can iterate over a `groupby` object, as we iterate we get pairs of `(name, group)`, where the `group` is either a `Series` or a `DataFrame`, depending on whether the `groupby` object is a `SeriesGroupBy` (as above) or a `DataFrameGroupBy` (see below).

Something quirky to note about the interaction between the grouped object and the for loop structure below: we're going to define variables `name` and `group` as being things in `grouped`.  But there are no `name` or `group` attributes associated with the `grouped` object.  

In [ ]:
for name, group in grouped:
    print('Name:', name)
    display(group)

We can group on multiple keys, and the result is grouping by tuples:

In [ ]:
g2 = df['data1'].groupby([df['key1'], df['key2']])
g2

In [ ]:
g2.groups

Now we have a groupby object that has tuples as the keys.  

In [ ]:
g2.mean()

### Aside (did not do in lecture)
We can also group the entire dataframe -- not just one column of it -- on a single key. This results in a `DataFrameGroupBy` object as the result:

In [ ]:
k1g = df.groupby('key1')
k1g

In [ ]:
k1g.groups

That output actually looks a lot like the output when we were only grouping one of the columns of the dataframe.  But there is actually more information in the group itself.  

In [ ]:
for name, group in k1g:
    print('Name:', name)
    display(group)

The contents of each group is another dataframe.

In [ ]:
k1g.mean()

Where did column `key2` go in the mean above? It's a *nuisance column*, which gets automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

### Aside (did not do in lecture): Grouping over a different dimension

Above, we've been grouping data along the rows, using column keys as our selectors.  

But we can also group along the *columns*, 

What's even more cool?  We can group by *data type*.

Here we'll group along columns, by data type:

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    display(group)

## Using groupby to re-ask our question 
*(did not do this in lecture, instead did CAISO forecasting error example)*
Which hour had the lowest average wind production?

In [ ]:
cds = pd.read_csv('CAISO_2017to2018_stack.csv', index_col= 0)

In [ ]:
cds.head()

It will help to have a column of hour of day values:

In [ ]:
cds_time = pd.to_datetime(cds.index)
type(cds_time)

Let's add that list of values into the data frame.

In [ ]:
cds['hour'] = cds_time.hour

In [ ]:
cds.head(10)

### Q: What groupby syntax would you use to arrange the data...
...so that you can examine production by hour and source?

See if you can do it yourself: we want to group MWh values by source AND hour.

In [ ]:
cds_grouped = cds['MWh'].groupby([cds['Source'],cds['hour']])

### Q: How to get *all* the means for all sources and hours?

Didn't need to do any fancy logical indexing or looping!

In [ ]:
cds_grouped.mean()

Now it would be nice to see that information in a dataframe, wouldn't it?

In [ ]:
averages = pd.DataFrame(cds_grouped.mean())

In [ ]:
averages

And lo and behold, we have a multilevel index for the rows!

In [ ]:
averages.loc[('WIND TOTAL',),:]

But now we can look at other sources

In [ ]:
averages.loc[('SMALL HYDRO',),:]

Let's plot:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(averages.loc[('SMALL HYDRO',),:]);

In [ ]:
plt.plot(averages.loc[('GEOTHERMAL',),:]);

In [ ]:
plt.plot(averages.loc[('SOLAR PV',),:]);